In [1]:
#!pip install simple_fpa

In [2]:
%%capture out
! cd ..; pip install .

In [3]:
print((out.stdout.split('\n')[-2]))

In [4]:
from simple_fpa import Model, Simulator
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import rcParams
from scipy.stats import beta

rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Arial"],
    "lines.linewidth": 1,
    "figure.dpi":200
})

import warnings
warnings.filterwarnings("ignore")

In [11]:
sample_size = 100000
smoothing_rate = 0.3
trim_percent = 5
eps = 0.05
frec = {3:1}

In [12]:
rv = beta(5,2)

In [13]:
sim = Simulator(sample_size, smoothing_rate, trim_percent, frec, rv.pdf, rv.ppf, eps, reflect = True)
sim.calibrate()
sim.make_true()

In [14]:
%time sim.simulate_uni(draws = 10000, nominal_coverage = 95)

CPU times: user 883 ms, sys: 796 ms, total: 1.68 s
Wall time: 53.7 s


In [ ]:
%time sim.simulate_dgp(draws = 10000, nominal_coverage = 95)

In [ ]:
for x,y in zip(['q','v','bs','rev','ts'],sim.cov):
    print('{:3} : {:}'.format(x,y))